In [48]:
import pandas as pd
import numpy as np
import requests
import pickle
import random
from itertools import combinations
from datetime import datetime
import json
import os
from tqdm import tqdm

In [49]:
#get list of tournaments
tournaments = [folder for folder in os.listdir('tournaments') if os.path.isdir(os.path.join('tournaments', folder))]
touraments = sorted(tournaments)

In [50]:
#get list of players
with open('metadata/playeridtoname.json', 'r') as json_file:
    player_dict = json.load(json_file)
playerids = list(player_dict.keys())

In [51]:
playerids

['40003218',
 '40001638',
 '40002651',
 '40000965',
 '40000216',
 '40000432',
 '40002471',
 '40001274',
 '40000152',
 '40000194',
 '40000638',
 '40001686',
 '40000430',
 '40000615',
 '40002785',
 '40000003',
 '40000596',
 '40000509',
 '40001199',
 '40000423',
 '40000764',
 '40004287',
 '40000585',
 '40002605',
 '40001424',
 '40000270',
 '40000004',
 '40000314',
 '40000540',
 '40002348',
 '40000011',
 '40000387',
 '40000946',
 '40001189',
 '40001414',
 '40002486',
 '40003609',
 '40000634',
 '40001971',
 '40000013',
 '40002026',
 '40001232',
 '40002474',
 '40002249',
 '40001447',
 '40000045',
 '40000561',
 '40003242',
 '40001685',
 '40000584',
 '40000659',
 '40000017',
 '40000647',
 '40000695',
 '40002343',
 '40001353',
 '40001870',
 '40003667',
 '40004729',
 '40003616',
 '40001680',
 '40001224',
 '40003684',
 '40002578',
 '40003346',
 '40001053',
 '40002573',
 '40000014',
 '40002645',
 '40000657',
 '40003612',
 '40000016',
 '40000763',
 '40001313',
 '40000010',
 '40001434',
 '40000632',

In [52]:
#list of elo calculatios
#round: k = 0.5, 1, 5, 10
#hole: k = 0.05, 0.1, 0.5, 1
#if players > 100, try increasing by factors of 1.25 and 1.5 in round 3

In [53]:
#what do we want to cache?
#for each player and each round, cache the updated elo and perf rating for that round
#structure - ratings -> tournament -> player -> round -> dataframe START BY CREATING TOURNS IN RATINGS
#col1 is updated elo, col2 is perf rating, rows are the different elos
#also: current -> dataframe with updated elos. rows are players, cols are elos
#also: current -> dataframe with rows as tournaments, columns as elos, booleans for calced

In [54]:
for t in tournaments:
    os.makedirs(f'ratings/{t}')

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'ratings/472'

In [55]:
ratings_calced = pd.DataFrame(index=tournaments, columns=['round_5_1'])
ratings_calced.fillna(0, inplace=True)
pd.to_pickle(ratings_calced, 'current/ratings_calced.pkl')

In [56]:
current_ratings = pd.DataFrame(index=playerids, columns=['round_5_1'])
current_ratings.fillna(0, inplace=True)
pd.to_pickle(current_ratings, 'current/current_ratings.pkl')

In [57]:
playerids

['40003218',
 '40001638',
 '40002651',
 '40000965',
 '40000216',
 '40000432',
 '40002471',
 '40001274',
 '40000152',
 '40000194',
 '40000638',
 '40001686',
 '40000430',
 '40000615',
 '40002785',
 '40000003',
 '40000596',
 '40000509',
 '40001199',
 '40000423',
 '40000764',
 '40004287',
 '40000585',
 '40002605',
 '40001424',
 '40000270',
 '40000004',
 '40000314',
 '40000540',
 '40002348',
 '40000011',
 '40000387',
 '40000946',
 '40001189',
 '40001414',
 '40002486',
 '40003609',
 '40000634',
 '40001971',
 '40000013',
 '40002026',
 '40001232',
 '40002474',
 '40002249',
 '40001447',
 '40000045',
 '40000561',
 '40003242',
 '40001685',
 '40000584',
 '40000659',
 '40000017',
 '40000647',
 '40000695',
 '40002343',
 '40001353',
 '40001870',
 '40003667',
 '40004729',
 '40003616',
 '40001680',
 '40001224',
 '40003684',
 '40002578',
 '40003346',
 '40001053',
 '40002573',
 '40000014',
 '40002645',
 '40000657',
 '40003612',
 '40000016',
 '40000763',
 '40001313',
 '40000010',
 '40001434',
 '40000632',

In [45]:
init_dict = {playerid: 1500 + random.choice([1, -1]) for playerid in playerids}
elo_round_point5_1 = init_dict
elo_round_point5_1quarter = init_dict
elo_round_point5_1half = init_dict
elo_round_1_1 = init_dict
elo_round_1_1quarter = init_dict
elo_round_1_1half = init_dict
elo_round_5_1 = init_dict
elo_round_5_1quarter = init_dict
elo_round_5_1half = init_dict
elo_round_10_1 = init_dict
elo_round_10_1quarter = init_dict
elo_round_10_1half = init_dict

In [46]:
elo_hole_point05_1 = {}
elo_hole_point05_1quarter = {}
elo_hole_point05_1half = {}
elo_hole_point1_1 = {}
elo_hole_point1_1quarter = {}
elo_hole_point1_1half = {}
elo_hole_point5_1 = {}
elo_hole_point5_1quarter = {}
elo_hole_point5_1half = {}
elo_hole_1_1 = {}
elo_hole_1_1quarter = {}
elo_hole_1_1half = {}

In [47]:
#try with max elo change
#round elos
k=0.7
for tournament in tqdm(tournaments):
    players = [folder for folder in os.listdir(f'tournaments/{tournament}')
               if os.path.isdir(os.path.join(f'tournaments/{tournament}', folder))]
#     if not os.path.exists(f'ratings/{tournament}'):
#         # Create the directory
#         os.makedirs(f'ratings/{tournament}')
#         for player in players:
#             if not os.path.exists(f'ratings/{tournament}/{player}'):
#                 os.makedirs(f'ratings/{tournament}/{player}')
#     else:
#         continue
    combos = list(combinations(players, 2))
    random.shuffle(combos)
    for r in range(4):
        preround_ratings = elo_round_5_1.copy()
        #print('hi')
        
        players_in_round = []
        random.shuffle(combos)
        opp_rating = {}
        winloss = {}
        games = {}
        for combo in combos:
            #print(combo)
            if not os.path.exists(f'tournaments/{tournament}/{combo[0]}'):
                #print('continue')
                continue
            if not os.path.exists(f'tournaments/{tournament}/{combo[1]}'):
                continue
            
            #print(players_in_round)
            a_path = f'tournaments/{tournament}/{combo[0]}/scorecard.pkl'
            b_path = f'tournaments/{tournament}/{combo[1]}/scorecard.pkl'
            

            with open(a_path, 'rb') as file:
                a_scorecard = pickle.load(file)
            with open(b_path, 'rb') as file:
                b_scorecard = pickle.load(file)

            if len(b_scorecard) == 0 or len(a_scorecard) == 0:
                #print('1')
                continue
            if len(a_scorecard) < r + 1 or len(b_scorecard) < r + 1:
                #print('2')
                continue
            if len(a_scorecard[r])<18 or len(b_scorecard[r])<18:
                #print('3')
                continue
            if len(a_scorecard[r]) == 0 or len(b_scorecard[r]) == 0:
                #print('3')
                continue
            if None in a_scorecard[r] or None in b_scorecard[r]:
                continue
                
            if combo[0] not in players_in_round:
                players_in_round.append(combo[0])
            if combo[1] not in players_in_round:
                players_in_round.append(combo[1])
            
            R_a = elo_round_5_1[combo[0]]
            R_b = elo_round_5_1[combo[1]]

            E_a = 1 / (1 + 10**((R_b - R_a)/480))
            E_b = 1 - E_a


            #print(a_scorecard[r])
            #print(b_scorecard[r])
            a_score = sum(a_scorecard[r])
            b_score = sum(b_scorecard[r])

            
            if combo[0] in opp_rating:
                opp_rating[combo[0]].append(R_b)
            else:
                opp_rating[combo[0]] = []
            if combo[1] in opp_rating:
                opp_rating[combo[1]].append(R_a)
            else:
                opp_rating[combo[1]] = []
                
            if combo[0] in games:
                games[combo[0]] += 1
            else:
                games[combo[0]] = 1
            if combo[1] in games:
                games[combo[1]] += 1
            else:
                games[combo[1]] = 1
    
            
            
            if a_score < b_score:
                a_result = 1
                b_result = 0
            elif b_score < a_score:
                b_result = 1
                a_result = 0
            else:
                a_result = 0.5
                b_result = 0.5
                
            
            if combo[0] in winloss:
                winloss[combo[0]] += a_result
            else:
                winloss[combo[0]] = 0
            if combo[1] in winloss:
                winloss[combo[1]] += b_result
            else:
                winloss[combo[1]] = 0    


            #print(R_a, k, a_result, E_a)
            R_a = R_a + k*(a_result - E_a)
            R_b = R_b + k*(b_result - E_b)
            #print(a_adjust)

            elo_round_5_1[combo[0]] = R_a
            elo_round_5_1[combo[1]] = R_b
            
#         for key, v in elo_round_5_1.items():
#             rating_diff = v - preround_ratings[key]
#             if rating_diff > 30:
#                 elo_round_5_1[key] = preround_ratings[key] + 30
#             if rating_diff < -30:
#                 elo_round_5_1[key] = preround_ratings[key] - 30
        #calc perf ratings
        elos_calced = pd.read_pickle('current/ratings_calced.pkl')
        current_elos = pd.read_pickle('current/current_ratings.pkl')
        for player in players_in_round:
            #print(players_in_round)
            #print(games)
            tot_games = games[player]
            tot_score = winloss[player]
            tot_opp_rating = sum(opp_rating[player])

            perf_rating = (tot_opp_rating + 400*tot_score) / tot_games

            path = f'ratings/{tournament}/{player}'
            if not os.path.exists(path):
                os.makedirs(path)
            
            player_round_rating_dict = {'elo_round_5_1':elo_round_5_1[player], 'perfelo_round_5_1':perf_rating}
            
            elos_calced.loc[tournament, 'elo_round_5_1'] = 1
            current_elos.loc[player, 'elo_round_5_1'] = elo_round_5_1[player]
            
            pd.to_pickle(elos_calced, 'current/ratings_calced.pkl')
            pd.to_pickle(current_elos, 'current/current_ratings.pkl')
            
            with open(f'ratings/{tournament}/{player}/{r}.json', 'w') as json_file:
                json.dump(player_round_rating_dict, json_file)
        
    
    

100%|████████████████████████████████████████████████| 88/88 [17:53<00:00, 12.19s/it]


In [23]:
def add_round_elo(tournaments, players, k, maxadjust, roundadjust, newkfactor):
    rating_name = f'elo_round_{k}_{newkfactor}_{roundadjust}'
    elos_calced = pd.read_pickle('current/ratings_calced.pkl')
    current_ratings = pd.read_pickle('current/current_ratings.pkl')
    rating_dict = {playerid: 1500 + random.choice([1, -1]) for playerid in playerids}
    for tournament in tournaments:
        combos = list(combinations(players, 2))
        random.shuffle(combos)
        for r in range(4):
            #print('hi')

            players_in_round = []
            random.shuffle(combos)
            opp_rating = {}
            winloss = {}
            games = {}
            for combo in combos:
                #print(combo)
                if not os.path.exists(f'tournaments/{tournament}/{combo[0]}'):
                    #print('continue')
                    continue
                if not os.path.exists(f'tournaments/{tournament}/{combo[1]}'):
                    continue

                #print(players_in_round)
                a_path = f'tournaments/{tournament}/{combo[0]}/scorecard.pkl'
                b_path = f'tournaments/{tournament}/{combo[1]}/scorecard.pkl'


                with open(a_path, 'rb') as file:
                    a_scorecard = pickle.load(file)
                with open(b_path, 'rb') as file:
                    b_scorecard = pickle.load(file)

                if len(b_scorecard) == 0 or len(a_scorecard) == 0:
                    #print('1')
                    continue
                if len(a_scorecard) < r + 1 or len(b_scorecard) < r + 1:
                    #print('2')
                    continue
                if len(a_scorecard[r])<18 or len(b_scorecard[r])<18:
                    #print('3')
                    continue
                if len(a_scorecard[r]) == 0 or len(b_scorecard[r]) == 0:
                    #print('3')
                    continue
                if None in a_scorecard[r] or None in b_scorecard[r]:
                    continue

                if combo[0] not in players_in_round:
                    players_in_round.append(combo[0])
                if combo[1] not in players_in_round:
                    players_in_round.append(combo[1])

                R_a = rating_dict[combo[0]]
                R_b = rating_dict[combo[1]]

                E_a = 1 / (1 + 10**((R_b - R_a)/480))
                E_b = 1 - E_a


                #print(a_scorecard[r])
                #print(b_scorecard[r])
                a_score = sum(a_scorecard[r])
                b_score = sum(b_scorecard[r])


                if combo[0] in opp_rating:
                    opp_rating[combo[0]].append(R_b)
                else:
                    opp_rating[combo[0]] = []
                if combo[1] in opp_rating:
                    opp_rating[combo[1]].append(R_a)
                else:
                    opp_rating[combo[1]] = []

                if combo[0] in games:
                    games[combo[0]] += 1
                else:
                    games[combo[0]] = 1
                if combo[1] in games:
                    games[combo[1]] += 1
                else:
                    games[combo[1]] = 1



                if a_score < b_score:
                    a_result = 1
                    b_result = 0
                elif b_score < a_score:
                    b_result = 1
                    a_result = 0
                else:
                    a_result = 0.5
                    b_result = 0.5


                if combo[0] in winloss:
                    winloss[combo[0]] += a_result
                else:
                    winloss[combo[0]] = 0
                if combo[1] in winloss:
                    winloss[combo[1]] += b_result
                else:
                    winloss[combo[1]] = 0    


                a_adjust = k*(a_result - E_a)
                b_adjust = k*(b_result - E_b)
                if a_adjust >= 0:
                    a_adjust = min(a_adjust, 30)
                else:
                    a_adjust = max(a_adjust, -30)
                if b_adjust >= 0:
                    b_adjust = min(b_adjust, 30)
                else:
                    b_adjust = max(b_adjust, -30)
                R_a = R_a + a_adjust
                R_b = R_b + b_adjust

                rating_dict[combo[0]] = R_a
                rating_dict[combo[1]] = R_b

            #calc perf ratings
            elos_calced = pd.read_pickle('current/ratings_calced.pkl')
            current_elos = pd.read_pickle('current/current_ratings.pkl')
            for player in players_in_round:
                #print(players_in_round)
                #print(games)
                tot_games = games[player]
                tot_score = winloss[player]
                tot_opp_rating = sum(opp_rating[player])

                perf_rating = (tot_opp_rating + 400*tot_score) / tot_games

                path = f'ratings/{tournament}/{player}'
                if not os.path.exists(path):
                    os.makedirs(path)

                player_round_rating_dict = {rating_name:elo_round_5_1[player], f'perf{rating_name}':perf_rating}

                elos_calced.loc[tournament, rating_name] = 1
                current_elos.loc[player, rating_name] = rating_dict[player]

                pd.to_pickle(elos_calced, 'current/ratings_calced.pkl')
                pd.to_pickle(current_elos, 'current/current_ratings.pkl')

                with open(f'ratings/{tournament}/{player}/{r}.json', 'w') as json_file:
                    json.dump(player_round_rating_dict, json_file)

In [29]:
pd.read_pickle('current/ratings_calced.pkl')

,round_5_1,elo_round_5_1
472,0,1.0
473,0,1.0
474,0,1.0
475,0,1.0
476,0,1.0
...,...,...
559,0,1.0
560,0,1.0
561,0,1.0
562,0,1.0


In [30]:
players_in_round

['40000270',
 '40000596',
 '40001424',
 '40001686',
 '40000430',
 '40002348',
 '40000615',
 '40004287',
 '40000314',
 '40000423',
 '40002785',
 '40001199',
 '40003218',
 '40000540',
 '40001274',
 '40000764',
 '40000216',
 '40000585',
 '40002471',
 '40000004',
 '40000152',
 '40002605',
 '40000194',
 '40002651',
 '40000003',
 '40000965',
 '40001638',
 '40000638',
 '40000432',
 '40000509']

In [31]:
len(players_in_round)

30

In [32]:
pd.read_pickle('tournaments/472/40002752/scorecard.pkl')

[[0, 0, 3, 0, -2, 0, 0, 0, -2, 0, -2, 0, 0, 0, -2, 0, 0, -2],
 [3, 0, 0, 0, -2, 0, 0, 0, 0, 0, 0, 0, 3, 6, -2, -2, 0, -2],
 [-2, 0, -2, 0, -2, 6, 0, 0, -2, -2, 0, 0, 0, -2, -2, 0, 0, 0],
 [9, 0, -2, 0, -2, -2, 0, 3, 6, -2, 0, 0, 0, 3, -2, 0, 0, -2]]

In [102]:
len([[]])

1